In [1]:
import os
import sys
sys.path.append('/home/mark/Documents/code/drone')

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
import networkx as nx
import gurobipy as gp
from gurobipy import GRB
from geopy.distance import lonlat, distance

from solver import *

main_seed = np.random.RandomState(10)
out_path = '/home/mark/Documents/code/drone/figures/temp/'

# data that we define by index
city_keys = ['SiouxFalls','Anaheim','ChicagoSketch']
city_names = ['Sioux Falls','Anaheim','Chicago']
n = [24,416,933]
m = [76,914,2950]
trips = [360600,104694,1260907]
centers = [13,13,13]

# data without chiacgo
# city_keys = ['SiouxFalls','Anaheim']
# city_names = ['Sioux Falls','Anaheim']
# n = [24,416]
# m = [76,914]
# trips = [360600,104694]
# centers = [13,13,13]

# city_keys = ['SiouxFalls','Anaheim']
# city_names = ['Sioux Falls','Anaheim']
# trips = [360600,104694,184679]
# n = [24,416,1020]
# m = [76,914,2522]
# centers = [13,13,13,13]

data = {}
i = 0
for city in city_keys:
    data[city] = {}

    network_file = '/home/mark/Documents/code/drone/tpnt/%s/%s_net.tntp'%(city,city)
    flow_file = '/home/mark/Documents/code/drone/tpnt/%s/%s_flow.tntp'%(city,city)

    net = pd.read_csv(network_file, skiprows=8, sep='\t')
    trimmed = [s.strip().lower() for s in net.columns]
    net.columns = trimmed
    net.drop(['~', ';'], axis=1, inplace=True)
    flow = pd.read_csv(flow_file, sep='\t')
    trimmed = [s.strip().lower() for s in flow.columns]
    flow.columns = trimmed
    
    # save data
    data[city]['net'] = net
    data[city]['flows'] = flow
    data[city]['name'] = city_names[i]
    data[city]['n'] = n[i]
    data[city]['m'] = m[i]
    data[city]['trips'] = trips[i]
    data[city]['node_map'] = lambda node: (node < 13)*node + (node == 13)*0 + (node > 13)*(node-1)
    i += 1


In [2]:
import numpy as np
D = np.ones((4,4))

D[:,0] /= 4
D

array([[0.25, 1.  , 1.  , 1.  ],
       [0.25, 1.  , 1.  , 1.  ],
       [0.25, 1.  , 1.  , 1.  ],
       [0.25, 1.  , 1.  , 1.  ]])

In [3]:
# lets look at the volume on the road and choose cutoff for lanes
for city in city_keys:
    lanes = []
    print(city)
    # data[city]['net']['capacity'].hist()
    # plt.show()
    # print(city)
    # data[city]['net']['free_flow_time'].hist()
    # plt.show()
    # data[city]['net']['length'].hist()
    # plt.show()
    # print(min(data[city]['net']['free_flow_time']))
    # plt.show()
    # print(np.where(data[city]['net']['length']<1e-19))
    
    for vol in data[city]['net']['capacity']:
        if vol > data[city]['net']['capacity'].median():
            lanes.append(3)
        else: lanes.append(2)
    data[city]['edge_lanes'] = lanes



SiouxFalls
Anaheim
ChicagoSketch


In [4]:
DRONE_SCALE = 1 # scale drone latency compared to shortest path latency
# CAP_SCAPE = 1 # scales capacity compared to flow on edge for solution
CUTOFF = 2  # num of paths to consider for ea OD pair

omega = {2:np.array([15.75812964, 0.02109056]), 
         3:np.array([4.26392855, 0.06173418]),
         4:np.array([1.91730372, 0.05962975])}

net_params = {}
optim_params = {}

for city in city_keys:
    # pre load for syntax 
    n = data[city]['n']
    m = data[city]['m']
    net = data[city]['net']
    edge_lanes = data[city]['edge_lanes']
    flows = data[city]['flows']
    node_map = data[city]['node_map']
    # adjecency matrix 
    A = np.zeros((n,n),dtype=int)
    # edge capacity equal to flow of solution * scale
    capacity = np.zeros((n,n),dtype=int)
    # lanes (2,3) (divide in half)
    lanes = np.zeros((n,n),dtype=int)
    # latency on edges
    l_road = np.zeros((n,n),dtype=int)
    # car flows
    flow_c = np.zeros((n,n),dtype=int)
    # if city == 'ChicagoSketch':
    #     for o, d, vol, length, e in zip(net['init_node'],net['term_node'],net['capacity'],net['free_flow_time'], net.index):
    #         # check if center node, and shift to first row/column
    #         o,d = node_map(o),node_map(d)
    #         A[o,d] = 1
    #         l_road[o,d] = length # arbitrary to convert, we will compare precentages
    #         capacity[o,d] = vol # 
    #         lanes[o,d] = edge_lanes[e]      
    # else:
    for o, d, vol, length, e in zip(net['init_node'],net['term_node'],net['capacity'],net['length'], net.index):
        # check if center node, and shift to first row/column
        o,d = node_map(o),node_map(d)
        A[o,d] = 1
        l_road[o,d] = length # arbitrary to convert, we will compare precentages
        capacity[o,d] = vol # 
        lanes[o,d] = edge_lanes[e]
    
    if city == 'ChicagoSketch':
        for o, d, flow in zip(flows['from'],flows['to'],flows['volume']):
            flow_c[(node_map(o),node_map(d))] = flow*2
    else:
        for o, d, flow in zip(flows['from'],flows['to'],flows['volume']):
            flow_c[(node_map(o),node_map(d))] = flow


    #### NET PARAMS ####
    net_params[city] = {}
    omega = {2:np.array([15.75812964, 0.02109056]), 
            3:np.array([4.26392855, 0.06173418]),
            4:np.array([1.91730372, 0.05962975])}

    demand = data[city]['trips']/72.12 #this gets 5000 demand for sioux falls
    net_params[city]['A'] = A
    net_params[city]['flow_c'] = flow_c
    net_params[city]['capacity'] = capacity
    net_params[city]['lanes'] = lanes
    net_params[city]['l_road'] = l_road
    net_params[city]['l_drone'] = None # for now
    net_params[city]['omega'] = omega  
    net_params[city]['cutoff'] = CUTOFF
    net_params[city]['Beta'] = data[city]['trips']
    net_params[city]['D_v'] = np.ones(n-1,dtype=int)*demand
    # net_params[city]['delivery_time'] n-1 0
    net_params[city]['pos'] = None # we will not render

    #### OPTIM PARAMS ####
    optim_params[city] = {}
    optim_params[city]['p_per_truck'] = 125 #125
    optim_params[city]['c_t'] = 30
    optim_params[city]['c_d'] = 0.5
    # assumes trucks are cheaper 
    c_min = optim_params[city]['c_t']*demand*(n-1)/optim_params[city]['p_per_truck']
    optim_params[city]['C_0'] = int(c_min*1.44928)

    optim_params[city]['MIP_gap'] = 0.00001
    optim_params[city]['allow_drones'] = True


In [5]:
keys = ['price', 'price_truck', 'price_drone', 'portion_truck','portion_drone','avg_s_l','latency_truck','latency_drone','avg_p_l']
solution = {}
for city in city_keys:
    solution[city] = () # tuple for easy data coupling
    print('-----'*8)
    print(city)
    print('-----'*8)
    data[city]['l_drone'] = []
    my_solver = net_solver(net_params[city])
    # first we need to get drone latency by scaling shortest path
    paths = create_paths(my_solver.G, cutoff=1) # just the shortest
    for path in paths:
        latency = 0
        for i in range(len(path)-1):
            edge = tuple(path[i:i+2])
            e = my_solver.edge_to_idx(edge)
            latency += my_solver.l_road[e] + np.dot([my_solver.omega[0][e],my_solver.omega[1][e]],[0,my_solver.flow_c[e]]) 
        data[city]['l_drone'].append(latency*DRONE_SCALE) # scale the latency
    # input latency after
    my_solver.l_drone = data[city]['l_drone']
    # save params
    #### OPTIMIZE ####
    for gamma in [1,0.5,0]:
        optim_params[city]['Gamma'] = gamma
        sol = my_solver.optimize(optim_params[city])
        solution[city] += ((gamma,sol),)



    # print(data[city]['n'])
    # print(data[city])
    

----------------------------------------
SiouxFalls
----------------------------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-26
Set parameter NonConvex to value 2
Set parameter MIPGap to value 1e-05
Set parameter NodefileStart to value 0.5
Set parameter Threads to value 8
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 8 threads
Optimize a model with 24 rows, 46 columns and 92 nonzeros
Model fingerprint: 0xc790aa19
Model has 610 quadratic objective terms
Coefficient statistics:
  Matrix range     [3e+01, 1e+02]
  Objective range  [1e-18, 2e-02]
  QObjective range [3e-08, 2e-05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 2e+04]

Continuous model is non-convex -- solving as a MIP

Presolve time: 0.00s
Presolved: 1153 rows, 657 columns, 2959 nonzeros
Presolved model has 46 quadratic constraint(s)
Presolved model has 564 bilinear constraint(s

In [ ]:
for city in city_keys:
    print('-----'*8)
    print(city)
    print('-----'*8)
    for gamma, sol in solution[city]:
        print('-----'*8)
        print('Gamma: ',gamma)
        print('Cost:', optim_params[city]['C_0'])
        print('-----'*8) 

        for key in keys:
            print(key,':\t',sol[key])
        gain_l = np.array(sol['gain_l'])
        print('min gain:\t',gain_l.min())
        print('median gain:\t',np.median(gain_l))
        print('max gain:\t',gain_l.max())

In [ ]:
# # l_road = np.array(data['ChicagoSketch']['net']['length'])
# l_road = my_solver.l_road
# # print(l_road.shape)
# # # print(l_drone.min(),l_drone.max())
# print(np.where(l_road < 1e-19))
# # print(np.where(data['ChicagoSketch']['net']['length'] < 1e-19))
# # print(l_drone)
# # print(my_solver.l_road.min(),my_solver.l_road.max())
# # print(np.min(my_solver.capacity))

# l_road.sort()
# print(l_road)

In [ ]:
# my_solver.l_road